In [1]:
!nvidia-smi

Sat Jul  4 07:14:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import math, re, os, random
import os
from tqdm import tqdm
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import pathlib
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import load_model
import math
from sklearn.model_selection import KFold, StratifiedKFold

print("TF version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

TF version 2.2.0


In [4]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!mkdir -p ~/.kaggle && cp '/content/drive/My Drive/kaggle.json' ~/.kaggle/ && chmod 600 ~/.kaggle/

In [6]:
!kaggle datasets download -d mylee2009/shopee-round-2-product-detection-challenge

100% 9.38G/9.38G [03:41<00:00, 46.1MB/s]
100% 9.38G/9.38G [03:41<00:00, 45.5MB/s]


In [7]:
!unzip -q shopee-round-2-product-detection-challenge.zip -d dataset

In [8]:
!rm shopee-round-2-product-detection-challenge.zip

In [9]:
#!cp -R '/content/drive/Shared drives/Jogjas Island [Shopee CL]/Week2/Pretrained' weight

In [5]:
MODEL_NAME = "EfficientNet-B4(380)_reget_trainable_1fold_V4"
IMG_HEIGHT = 380
IMG_WIDTH = 380
IMAGE_SIZE = (IMG_HEIGHT, IMG_WIDTH)
EPOCHS = 6
BATCH_SIZE = 40 * strategy.num_replicas_in_sync
SEED = 177013
SKIP_VALIDATION = False
SKIP_TRAINING = False
TTA_NUM = 1
ES_PATIENCE = 10
RLROP_PATIENCE = 10
DECAY_RATE = 0.5
DROPOUT_RATE = 0.2


random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [6]:
BASE_DIR = "dataset/" 
TRAIN_DIR = pathlib.Path(BASE_DIR + "train/train/")
TEST_DIR = pathlib.Path(BASE_DIR + "test/test/")

CLASS_NAMES = np.sort(np.array([item.name for item in TRAIN_DIR.glob('*')]))
#list_ds = tf.data.Dataset.list_files(str(TRAIN_DIR/'*/*'))
#IMAGE_COUNT = len(list(TRAIN_DIR.glob('*/*.jpg')))

In [7]:
CLASSES = ['short dress',
  'long dress',
  'women top',
  'women outerwear',
  'women pants',
  'ring',
  'earring',
  'hat',
  'purse',
  'bag / case',
  'casing HP',
  'iphone ',
  'clock',
  'baby gear',
  'rice cooker',
  'coffee',
  'women flatshoes',
  'heels',
  'household electronic',
  'storage device',
  'berdoom furniture',
  'tennis/badminton equipment',
  'bicycle equipment',
  'handwear / footwear',
  'watches',
  'belt ',
  'audio equipment',
  'toy vehicle',
  'man outerwear',
  'man pants',
  'man shoes',
  'snack',
  'PPE',
  'antiseptic',
  'skincare / cosmetic',
  'perfume',
  'hygiene equipment',
  'laptop & accessories',
  'tableware',
  'vase',
  'bathroom equipment',
  'living room furniture']

In [8]:
testdf = pd.read_csv("./dataset/test.csv",dtype=str)
testdf.head()

,filename,category
0,fd663cf2b6e1d7b02938c6aaae0a32d2.jpg,43
1,c7fd77508a8c355eaab0d4e10efd6b15.jpg,43
2,127f3e6d6e3491b2459812353f33a913.jpg,43
3,5ca4f2da11eda083064e6c36f37eeb81.jpg,43
4,46d681a542f2c71be017eef6aae23313.jpg,43


In [9]:
traindir_df = pd.read_csv("/content/drive/Shared drives/Jogjas Island [Shopee CL]/Week2/Training/train_with_dir.csv",dtype=str)
traindir_df = traindir_df.sample(frac=1.0).reset_index(drop=True)
traindir_df.head()

Y_train = traindir_df[["category"]]

skf = StratifiedKFold(n_splits = 5, random_state = SEED, shuffle = True)
#finetunedf.describe()

In [10]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

'''
                                                                    rotation_range=40,
                                                                    width_shift_range=0.2,
                                                                    height_shift_range=0.2,
                                                                    shear_range=0.2,
                                                                    zoom_range=0.2,
                                                                    horizontal_flip=True,
                                                                    fill_mode="nearest",
'''
train_ds = train_generator.flow_from_directory(directory=str(TRAIN_DIR),
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                classes = list(CLASS_NAMES),
                                                seed = SEED)
                                                #subset='training')
'''
validation_ds = train_generator.flow_from_directory(directory=str(TRAIN_DIR),
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                classes = list(CLASS_NAMES),
                                                seed = SEED,
                                                subset='validation')

VAL_SPLIT = 0.19
train_take = np.ceil(VAL_SPLIT * IMAGE_COUNT)

validation_ds = train_ds.skip(train_take)
train_ds = train_ds.take(train_take)

STEPS_PER_EPOCH = train_take // BATCH_SIZE
VAL_STEPS_PER_EPOCH = (IMAGE_COUNT-train_take) // BATCH_SIZE
'''

STEPS_PER_EPOCH = train_ds.samples // BATCH_SIZE
#VAL_STEPS_PER_EPOCH = validation_ds.samples // BATCH_SIZE

Found 105392 images belonging to 42 classes.


In [11]:
def get_training_dataset():
  return train_ds
def get_validation_dataset():
  return validation_ds
def get_test_dataset():
  return test_ds  

In [12]:
def get_callback():
  filepath = "/content/drive/Shared drives/Jogjas Island [Shopee CL]/Week2/Pretrained_Fix/"+MODEL_NAME+"-{epoch:02d}.hdf5"
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto')

  es = EarlyStopping(monitor = 'loss', mode = 'min', patience = ES_PATIENCE, restore_best_weights = True, verbose = 1)
  rlrop = ReduceLROnPlateau(monitor = 'loss', mode = 'min', patience = RLROP_PATIENCE, 
                          factor = DECAY_RATE, min_lr = 1e-6, verbose = 1)
  return [model_checkpoint_callback, rlrop]

In [13]:
!pip install -q efficientnet
import efficientnet.tfkeras as efn

In [14]:
model = tf.keras.models.load_model("/content/drive/Shared drives/Jogjas Island [Shopee CL]/Week2/Pretrained_Fix/EfficientNet-B4(380)_reget_trainable_1fold_V3-03.hdf5")
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b4 (Model)      (None, 16, 16, 1792)      17673816  
_________________________________________________________________
gap (GlobalAveragePooling2D) (None, 1792)              0         
_________________________________________________________________
dropout_out (Dropout)        (None, 1792)              0         
_________________________________________________________________
fc_out (Dense)               (None, 42)                75306     
Total params: 17,749,122
Trainable params: 4,662,938
Non-trainable params: 13,086,184
_________________________________________________________________


In [ ]:
history = model.fit(train_ds, 
                    steps_per_epoch=STEPS_PER_EPOCH, 
                    epochs=EPOCHS, 
                    #validation_data = validation_ds,
                    #validation_steps = VAL_STEPS_PER_EPOCH,
                    verbose=1,
                    max_queue_size = 80,
                    workers = 128,
                    callbacks = get_callback())

Epoch 1/6
2634/2634 [==============================] - ETA: 0s - loss: 0.0031 - cat_acc: 0.9988 - auc: 1.0000 - top5: 1.0000
Epoch 00001: saving model to /content/drive/Shared drives/Jogjas Island [Shopee CL]/Week2/Pretrained_Fix/EfficientNet-B4(380)_reget_trainable_1fold_V4-01.hdf5
2634/2634 [==============================] - 1078s 409ms/step - loss: 0.0031 - cat_acc: 0.9988 - auc: 1.0000 - top5: 1.0000 - lr: 2.7563e-04
Epoch 2/6
2634/2634 [==============================] - ETA: 0s - loss: 0.0296 - cat_acc: 0.9547 - auc: 0.9996 - top5: 0.9985
Epoch 00002: saving model to /content/drive/Shared drives/Jogjas Island [Shopee CL]/Week2/Pretrained_Fix/EfficientNet-B4(380)_reget_trainable_1fold_V4-02.hdf5
2634/2634 [==============================] - 1060s 403ms/step - loss: 0.0296 - cat_acc: 0.9547 - auc: 0.9996 - top5: 0.9985 - lr: 2.7563e-04
Epoch 3/6
1184/2634 [============>.................] - ETA: 9:47 - loss: 0.0261 - cat_acc: 0.9591 - auc: 0.9997 - top5: 0.9991

In [ ]:
model.save("/content/drive/Shared drives/Jogjas Island [Shopee CL]/Week2/Pretrained_Fix/{0}-Final.hdf5".format(MODEL_NAME))

In [ ]:
class_names = sorted(train_ds.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
print(class_names)
for image_batch, label_batch in train_ds:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

In [ ]:
preds_tta = []

test_ds = test_generator.flow_from_dataframe(dataframe = testdf,
                                              directory=str(TEST_DIR),
                                              x_col="filename",
                                              y_col=None,
                                              shuffle=False,
                                              class_mode=None,
                                              seed = SEED,
                                              target_size=IMAGE_SIZE)
for image_test in test_ds:
  print("Image batch shape: ", image_test.shape)
  break

test_ds.reset()
predicted_test = model.predict(test_ds, 
                                  verbose=1,
                                  max_queue_size=72,
                                  workers=128)
preds_tta.append(predicted_test)

In [ ]:
predicted_batch = model.predict(image_batch, 
                                 verbose=1,
                                 max_queue_size=72,
                                 workers=128)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

label_id = np.argmax(label_batch, axis=-1)
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  color = "green" if predicted_id[n] == label_id[n] else "red"
  plt.title(CLASSES[predicted_label_batch[n].astype(int)], color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

In [ ]:
#final_pred = np.mean(preds_tta, axis=0)
predicted_test_id = np.argmax(predicted_test, axis=-1)
predicted_label_test = class_names[predicted_test_id]

In [ ]:
predicted_test

In [ ]:
labels = (train_ds.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_test_id]

In [ ]:
plt.figure(figsize=(16,14))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_test[n])
  color = "green"
  plt.title(CLASSES[predictions[n].astype(int)], color=color)
  plt.axis('off')

In [ ]:
test_ds.reset()
filename = test_ds.filenames
results0 = pd.DataFrame({"filename":filename,
                      "category":predictions})
results0.to_csv("results_{0}.csv".format(MODEL_NAME),index=False)